In [1]:
import ROOT

In [ ]:
# observable
x = ROOT.RooRealVar("x", "invariant mass", 110, 135,"GeV")
x.setBins(10)

In [3]:
# model for signal

mass = ROOT.RooRealVar("mass", "higgs mass", 130, 110, 150)
width = ROOT.RooRealVar("width", "higgs width", 4.1/2.35) 

alpha = ROOT.RooRealVar("alpha", "CB alpha", 0.6)
n = ROOT.RooRealVar("n", "CB tail", 20)

smodel = ROOT.RooCBShape("smodel", "signal model", x, mass, width, alpha, n)

In [4]:
# model for bkg

a1 = ROOT.RooRealVar("a1", "The a1 of background", -160, -200, -100)
a2 = ROOT.RooRealVar("a2", "The a2 of background", 2.7, 2, 4)
bmodel = ROOT.RooPolynomial("bmodel", "bkg", x, ROOT.RooArgList(a1,a2))

In [5]:
# composite model (rate + shape -> extended likelihood -> marked poisson model)
 
nsignal = ROOT.RooRealVar("nsig", "signal yield", 100, 0, 500)
nbackground = ROOT.RooRealVar("nbkg", "bkg yield", 100, 0, 500)
model = ROOT.RooAddPdf("model", 
                       "S+B model", 
                       ROOT.RooArgList(smodel, bmodel),
                       ROOT.RooArgList(nsignal, nbackground) )
                       

In [6]:
frame1 = x.frame()
model.plotOn(frame1)
model.plotOn(frame1, 
             ROOT.RooFit.Components("bmodel"), 
             ROOT.RooFit.LineStyle(ROOT.kDashed))

c = ROOT.TCanvas()
frame1.Draw()
c.Draw()

[#1] INFO:Plotting -- RooAbsPdf::plotOn(model) directly selected PDF components: (bmodel)
[#1] INFO:Plotting -- RooAbsPdf::plotOn(model) indirectly selected PDF components: ()


In [7]:
# load the data
data1 = ROOT.RooDataSet.read("higgs_4l.dat", ROOT.RooArgList(x))

data1.Print("v")

# remove "BlindState" variable from the dataset
data = data1.reduce(ROOT.RooArgSet(x))

data.Print("v")


[#1] INFO:DataHandling -- RooDataSet::read: reading file higgs_4l.dat
[#1] INFO:DataHandling -- RooDataSet::read: read 123 events (ignored 0 out of range events)
DataStore dataset (higgs_4l.dat)
  Contains 123 entries
  Observables: 
    1)           x = 133.437  L(110 - 135) B(10) // [GeV] "invariant mass"
    2)  blindState = Normal(idx = 0)
  "Blinding State"
DataStore dataset (higgs_4l.dat)
  Contains 123 entries
  Observables: 
    1)  x = 133.437  L(110 - 135) B(10) // [GeV] "invariant mass"


In [8]:
# fit
 
fit_results = model.fitTo(data, ROOT.RooFit.Save())

[#1] INFO:Minimization -- p.d.f. provides expected number of events, including extended term in likelihood.
[#1] INFO:Fitting -- RooAbsPdf::fitTo(model) fixing normalization set for coefficient determination to observables in data
[#1] INFO:Fitting -- using generic CPU library compiled with no vectorizations
[#1] INFO:Fitting -- Creation of NLL object took 1.48607 ms
[#1] INFO:Fitting -- RooAddition::defaultErrorLevel(nll_model_dataset) Summation contains a RooNLLVar, using its error level
[#1] INFO:Minimization -- RooAbsMinimizerFcn::setOptimizeConst: activating const optimization
Minuit2Minimizer: Minimize with max-calls 2500 convergence for edm < 1 strategy 1
Minuit2Minimizer : Valid minimum - status = 0
FVAL  = -96.0244009707144812
Edm   = 0.000399674053569116735
Nfcn  = 120
a1	  = -162.544	 +/-  68.4829	(limited)
a2	  = 2.68193	 +/-  1.59422	(limited)
mass	  = 124.399	 +/-  0.679381	(limited)
nbkg	  = 57.6493	 +/-  11.7784	(limited)
nsig	  = 65.3599	 +/-  12.0983	(limited)
[#1] IN

Info in <Minuit2>: MnSeedGenerator Computing seed using NumericalGradient calculator
Info in <Minuit2>: MnSeedGenerator Evaluated function and gradient in 245.275 μs
Info in <Minuit2>: MnSeedGenerator Initial state: FCN =      -42.62825752 Edm =       20559.18611 NCalls =     21
Info in <Minuit2>: NegativeG2LineSearch Doing a NegativeG2LineSearch since one of the G2 component is negative
Info in <Minuit2>: NegativeG2LineSearch Done after 99.102 μs
Info in <Minuit2>: MnSeedGenerator Negative G2 found - new state: 
  Minimum value : -78.72052167
  Edm           : 27.42260383
  Internal parameters:	[    -0.2013579208     -0.304692654    -0.2799583375    -0.6435011088    -0.6435011088]	
  Internal gradient  :	[   -0.02164934579   -0.02499049059      7.962463555      79.60839205      74.39159701]	
  Internal covariance matrix:
[[      37.266301              0              0              0              0]
 [              0      22.949275              0              0              0]
 [      

In [9]:
fit_results.Print("v")


  RooFitResult: minimized FCN value: -96.0244, estimated distance to minimum: 0.000434894
                covariance matrix quality: Full matrix, but forced positive-definite
                Status : MINIMIZE=0 HESSE=0 

    Constant Parameter    Value     
  --------------------  ------------
                 alpha    6.0000e-01
                     n    2.0000e+01
                 width    1.7447e+00

    Floating Parameter  InitialValue    FinalValue +/-  Error     GblCorr.
  --------------------  ------------  --------------------------  --------
                    a1   -1.6000e+02   -1.6254e+02 +/-  6.35e+01  <none>
                    a2    2.7000e+00    2.6819e+00 +/-  1.52e+00  <none>
                  mass    1.3000e+02    1.2440e+02 +/-  6.85e-01  <none>
                  nbkg    1.0000e+02    5.7649e+01 +/-  1.17e+01  <none>
                  nsig    1.0000e+02    6.5360e+01 +/-  1.20e+01  <none>



In [10]:
frame2 = x.frame()

data.plotOn(frame2)
model.plotOn(frame2)
model.plotOn(frame2, 
             ROOT.RooFit.Components("bmodel"),
             ROOT.RooFit.LineStyle(ROOT.kDashed)
             )

[#1] INFO:Plotting -- RooAbsPdf::plotOn(model) directly selected PDF components: (bmodel)
[#1] INFO:Plotting -- RooAbsPdf::plotOn(model) indirectly selected PDF components: ()


In [11]:
c = ROOT.TCanvas()
frame2.Draw()
c.Draw()

In [ ]:
# import all in workspace
w = ROOT.RooWorkspace("w")

w.Import(model)
w.Import(data)

False

[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooAddPdf::model
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooCBShape::smodel
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooRealVar::x
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooRealVar::mass
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooRealVar::width
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooRealVar::alpha
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooRealVar::n
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooRealVar::nsig
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooPolynomial::bmodel
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooRealVar::a1
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooRealVar::a2
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooRealVar::nbkg
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) i

In [13]:
# check 

w.Print()


RooWorkspace(w) w contents

variables
---------
(a1,a2,alpha,mass,n,nbkg,nsig,width,x)

p.d.f.s
-------
RooPolynomial::bmodel[ x=x coefList=(a1,a2) ] = 26064.4
RooAddPdf::model[ nsig * smodel + nbkg * bmodel ] = 12215.3/1
RooCBShape::smodel[ m=x m0=mass sigma=width alpha=alpha n=n ] = 1.48809e-06

datasets
--------
RooDataSet::dataset(x)



In [14]:
# write a model configuration , THIS IS FOR SIGNIFICANCE
# compute the excess of "nsig"

mc = ROOT.RooStats.ModelConfig("ModelConfig")
mc.SetWorkspace(w)
mc.SetPdf(w.pdf("model"))
mc.SetObservables(w.var("x"))

# constants
w.var("width").setConstant(True)
w.var("alpha").setConstant(True)
w.var("n").setConstant(True)

# parameters are nsig, nbkg, a1, a2, mass

mc.SetParametersOfInterest(w.var("nsig"))

# the other are nuisance define a list 
w.defineSet("nuis", "nbkg,a1,a2")
mc.SetNuisanceParameters(w.set("nuis"))

# special case: the mass, this is required for significance
# w.var("mass").setConstant(True)

mc.SetSnapshot(w.var("nsig"))

In [15]:
# write a model configuration , THIS IS FOR mass INTERVALS
# 

mc2 = ROOT.RooStats.ModelConfig("ModelConfig_mass")
mc2.SetWorkspace(w)
mc2.SetPdf(w.pdf("model"))
mc2.SetObservables(w.var("x"))

# parameters are nsig, nbkg, a1, a2, mass
mc2.SetParametersOfInterest(w.var("mass"))

# the other are nuisance, define a list 
w.defineSet("nuis_2", "nbkg,a1,a2,nsig")
mc2.SetNuisanceParameters(w.set("nuis_2"))


In [16]:
w.Import(mc)
w.Import(mc2)

w.Print("v")


RooWorkspace(w) w contents

variables
---------
(a1,a2,alpha,mass,n,nbkg,nsig,width,x)

p.d.f.s
-------
RooPolynomial::bmodel[ x=x coefList=(a1,a2) ] = 26064.4
RooAddPdf::model[ nsig * smodel + nbkg * bmodel ] = 12215.3/1
RooCBShape::smodel[ m=x m0=mass sigma=width alpha=alpha n=n ] = 1.48809e-06

datasets
--------
RooDataSet::dataset(x)

parameter snapshots
-------------------
ModelConfig__snapshot = (nsig=65.3599 +/- 11.9881)

named sets
----------
ModelConfig_NuisParams:(nbkg,a1,a2)
ModelConfig_Observables:(x)
ModelConfig_POI:(nsig)
ModelConfig__snapshot:(nsig)
ModelConfig_mass_NuisParams:(nbkg,a1,a2,nsig)
ModelConfig_mass_Observables:(x)
ModelConfig_mass_POI:(mass)
nuis:(nbkg,a1,a2)
nuis_2:(nbkg,a1,a2,nsig)

generic objects
---------------
RooStats::ModelConfig::ModelConfig
RooStats::ModelConfig::ModelConfig_mass



In [17]:
w.writeToFile("atlas_4ll.root")

False

### ROOT INTERACTIVE

```bash
$ root
```

#### SIGNIFICANCE (2 = asymptotic calculator, 3 = Profile Likelihood Ratio one-side)
```cpp
root[] .L $ROOTSYS/tutorials/roofit/roostats/StandardHypoTestDemo.C
root[] StandardHypoTestDemo("atlas_4ll.root", "w", "ModelConfig", "", "dataset", 2, 3)
root[] .q
```

#### 68% C.L. Profile Likelihood Interval 
(using `ModelConfig_mass` configuration)
```cpp
root[] .L $ROOTSYS/tutorials/roofit/roostats/StandardProfileLikelihoodDemo.C
root[] optPL.confLevel = 0.68
root[] StandardProfileLikelihoodDemo("atlas_4ll.root", "w", "ModelConfig_mass", "dataset")
root[] .q
```

#### 68% C.L. Feldman Cousins Interval 
(using `ModelConfig_mass` configuration)
The demo calculator in
`$ROOTSYS/tutorials/roofit/roostats/StandardFeldmanCousinsDemo.C`
has the confidence level hard-coded to 0.95.
We copied the file and manually changed the confidence level to 0.68.
```cpp
root[] .L StandardFeldmanCousinsDemo.C 
root[] StandardFeldmanCousinsDemo("atlas_4ll.root", "w", "ModelConfig_mass", "dataset")
root[] .q
```
